# Environment setup

In [35]:
pip install pyfaidx Bio matplotlib regex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 35.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [44]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pyfaidx import Fasta
from Bio.Seq import Seq
import regex as re

# === SETTINGS ===
genome = Fasta("../references/genome.fold.FINAL.fasta.noDuplicates_noMito.fasta")

# Motifs and allowed mismatches
motifs = [
    ("CACGTG", 0),  # strict match
    ("GTTAAGCAAATTAAATTTGATTCT", 6)  # allowed up to 6 substitutions
]

motif_regex = [
    re.compile(f"({motif}){{s<={n},i<=0,d<=0}}")
    for motif, n in motifs
]

# Target loci: (gene_name, chromosome, center_position, window)
targets = [
    ("test", "HiC_scaffold_1", 123456, 1000),
    ("SoxB1", "HiC_scaffold_9", 39576426, 40000),
]

os.makedirs("motif_plots", exist_ok=True)

summary = []

for gene, chrom, center, window in targets:
    region_start = center - window
    region_end = center + window

    print(f"\n🔍 Checking {gene} ({chrom}:{region_start}-{region_end})")

    try:
        seq_region = genome[chrom][region_start:region_end].seq.upper()
    except KeyError:
        print(f"❌ Chromosome '{chrom}' not found in genome FASTA.")
        continue

    rev_comp = str(Seq(seq_region).reverse_complement())
    hits = []

    for (motif, max_mismatch), rx in zip(motifs, motif_regex):
        # Forward strand
        for match in rx.finditer(seq_region, overlapped=True):
            mismatches = match.fuzzy_counts[0]
            hits.append((gene, chrom, "+", match.start() + region_start, motif, mismatches))

        # Reverse strand
        for match in rx.finditer(rev_comp, overlapped=True):
            mismatches = match.fuzzy_counts[0]
            hits.append((gene, chrom, "-", region_end - match.start() - 1, motif, mismatches))

    # === Save table ===
    df = pd.DataFrame(hits, columns=["gene", "chrom", "strand", "position", "motif", "mismatches"])
    tsv_file = f"motif_plots/{gene}_{chrom}_{center}_motif_hits.tsv"
    df.to_csv(tsv_file, sep="\t", index=False)

    if df.empty:
        print(f"⚠️ No motifs found for {gene}")
    else:
        print(df.to_string(index=False))

    summary.append((gene, chrom, len(df)))

    # === Plot ===
    fig, ax = plt.subplots(figsize=(10, 2))
    ax.hlines(0, region_start, region_end, color="gray", linewidth=2)
    ax.set_xlim(region_start, region_end)
    ax.set_ylim(-2, 2)
    ax.set_yticks([-1, 0, 1])
    ax.set_yticklabels(["Rev", "", "Fwd"])
    ax.set_xlabel(f"Genomic position on {chrom}")
    ax.set_title(f"{gene} ({chrom}:{region_start}-{region_end})")

    for _, _, strand, pos, motif, mismatches in hits:
        y = 1 if strand == "+" else -1
        label = f"{motif} (mm={mismatches})" if mismatches > 0 else motif
        ax.plot(pos, y, marker="^", color="blue" if strand == "+" else "red", markersize=8)
        ax.text(pos, y + 0.2 if strand == "+" else y - 0.4, label, ha="center", fontsize=7, rotation=45)

    plt.tight_layout()
    plt.savefig(f"motif_plots/{gene}_{chrom}_{center}.png")
    plt.close()

# === Summary Report ===
print("\n📊 Motif Hit Summary:")
for gene, chrom, count in summary:
    print(f" - {gene} ({chrom}): {count} hits")

print("\n✅ Done. All plots and tables saved to motif_plots/")



🔍 Checking test (HiC_scaffold_1:122456-124456)
⚠️ No motifs found for test

🔍 Checking SoxB1 (HiC_scaffold_9:39536426-39616426)
 gene          chrom strand  position                    motif  mismatches
SoxB1 HiC_scaffold_9      +  39539632                   CACGTG           0
SoxB1 HiC_scaffold_9      +  39564481                   CACGTG           0
SoxB1 HiC_scaffold_9      +  39564635                   CACGTG           0
SoxB1 HiC_scaffold_9      +  39570108                   CACGTG           0
SoxB1 HiC_scaffold_9      +  39600051                   CACGTG           0
SoxB1 HiC_scaffold_9      -  39600056                   CACGTG           0
SoxB1 HiC_scaffold_9      -  39570113                   CACGTG           0
SoxB1 HiC_scaffold_9      -  39564640                   CACGTG           0
SoxB1 HiC_scaffold_9      -  39564486                   CACGTG           0
SoxB1 HiC_scaffold_9      -  39539637                   CACGTG           0
SoxB1 HiC_scaffold_9      -  39603046 GTTAAGCA